In [1]:
from keras.models import Model
from keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
import import_ipynb
from reccnn import RecCNN
from comcnn import ComCNN

/home/chandu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


importing Jupyter notebook from reccnn.ipynb
importing Jupyter notebook from comcnn.ipynb


In [2]:
import matplotlib.image as mpimg
from PIL import Image
import os

def compressThis(x_input):
    # x_input dimension (num, x, y, c)
    # num - number of images
    num = x_input.shape[0]
    ans = []
    for i in range(num):
        x_single = x_input[i]
        # compressing single image
        mpimg.imsave(os.getcwd()+'/org.JPEG', x_single)
        tmp = Image.open(os.getcwd()+'/org.JPEG')
        tmp.save(os.getcwd()+'/com.JPEG',"JPEG", optimize=True, quality=65)
        out_single = mpimg.imread(os.getcwd()+'/com.JPEG')
        ans.append(out_single)
        
    return np.array(ans)
        

In [3]:
import os
import matplotlib.image as mpimg

loc = os.getcwd()+'/DSet-Color'
X = []
for i in os.listdir(loc):
    im_loc = loc + '/'+i
    img = mpimg.imread(im_loc)
    X.append(img)

X = np.array(X)
print(X.shape)

(199, 375, 500, 3)


In [4]:
x_train = X[:190,:,:,:]
x_test = X[190:,:,:,:]
print(x_train.shape)
print(x_test.shape)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train/=255
x_train*=2
x_train-=1
x_test/=255
x_test*=2
x_test-=1


(10, 375, 500, 3)
(1, 375, 500, 3)


In [34]:
from keras.layers import Input
from keras.callbacks import ModelCheckpoint
import tensorflow as tf


inp2 = Input(shape=(25,50,3))
rec_cnn = RecCNN(3)
model_reccnn = Model(inp2, rec_cnn.sisr(inp2))
model_reccnn.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
checkpointr = ModelCheckpoint(filepath='reccnn.weights.best.hdf5', save_best_only=True, verbose=1)
model_reccnn.summary()

inp1 = Input(shape=(375,500,3))
com_cnn = ComCNN(3)
model_comcnn = Model(inp1, model_reccnn(com_cnn.compact(inp1)))
model_comcnn.layers[4].trainable = False
model_comcnn.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
checkpointc = ModelCheckpoint(filepath='comcnn.weights.best.hdf5', save_best_only=True, verbose=1)
model_comcnn.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 25, 50, 3)    0                                            
__________________________________________________________________________________________________
interpolate_tensor_5 (Interpola (None, 375, 500, 3)  0           input_9[0][0]                    
__________________________________________________________________________________________________
conv2d_25 (Conv2D)              (None, 375, 500, 64) 4864        interpolate_tensor_5[0][0]       
__________________________________________________________________________________________________
conv2d_26 (Conv2D)              (None, 375, 500, 32) 18464       conv2d_25[0][0]                  
__________________________________________________________________________________________________
conv2d_27 

In [12]:
count = 100

while count > 100:
    
    # calculating cm using comcnn
    upto_comcnn = Model(model_comcnn.input, model_comcnn.layers[3].output)
    xm = upto_comcnn.predict(x_train)
    xm = xm*255
    xm = compressThis(xm)
    xm = xm/255
#     print(xm.shape)
    
    # train RecCNN
    model_reccnn.fit(x=xm, y=x_train, epochs=20, shuffle=True, verbose=1, batch_size=8, callbacks=[checkpointr])
    
    # train ComCNN
    model_comcnn.fit(x=x_train, y=x_train, epochs=20, shuffle=True, verbose=1, batch_size=8, callbacks=[checkpointc])
    count = count - 1

/home/chandu/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Epoch 1/1
10/10 [==============================] - 13s 1s/step - loss: 1.4910
Epoch 1/1
10/10 [==============================] - 8s 849ms/step - loss: 0.8300
